# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

This is a classification problem as we will be classifying students into two categories, one that need intervention and ones that don't, there is no middle ground or grey area that we can regress to. 

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [152]:
# Import libraries
import numpy as np
import pandas as pd

In [153]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns
print(student_data.head())
print(student_data.columns)

Student data read successfully!
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

   ...   internet romantic  famrel  freetime  goout Dalc Walc health absences  \
0  ...         no       no       4         3      4    1    1      3        6   
1  ...        yes       no       5         3      3    1    1      3        4   
2  ...        yes       no       4         3      2    2    3      3       10   
3  ...        yes      yes       3         2      2    1    1      5        2   
4  ...         no       no       4         3      2    1    2      5        4   

  passed  
0     no  
1 

Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [163]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = student_data[student_data['passed'] == 'yes'].shape[0]
n_failed = student_data[student_data['passed'] == 'no'].shape[0]
grad_rate = (265.0/395.0)*100
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [155]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label

print "**Feature column(s):**-\n{}".format(feature_cols)
print('\n')
print "**Target column:** \n{}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

**Feature column(s):**-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


**Target column:** 
passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   


### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [156]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [157]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.cross_validation import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X_all, 
                                                    y_all, 
                                                    test_size = num_test, 
                                                    random_state = 42)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What is the theoretical O(n) time & space complexity in terms of input size?
- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

The three models I am picking are:

1. Support Vector Machines 
2. Random Forest Trees
3. Gaussian Naive Bayes


### Support Vector Machines

- What is the theoretical O(n) time & space complexity in terms of input size?

    Standard SVM training has O(`n^3`) time and O(`n^2`) space complexities, where `n` is the training set size.

- What are the general applications of this model? What are its strengths and weaknesses?
    
    Some applications where it can be used are:
    
    In medical science to classify proteins.
    In image,text and hypertext categorization.

    Strengths:
    Training is relatively easy and it has no local optimal, unlike in neural networks.
    Effective in high dimensional spaces.
    Effective in cases where number of dimensions is greater than the number of samples.
    Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
    In addition to performing linear classification, SVMs can efficiently perform a non-linear classification using the kernel trick, implicitly mapping their inputs into high-dimensional feature spaces.
    Non-traditional data like strings and trees can be used as input to SVM, instead of feature vectors

    Weaknesses:
    If the number of features is much greater than the number of samples, the method is likely to give poor performances.
    It needs a “good” kernel function.
    Prone to overfitting.

- Given what you know about the data so far, why did you choose this model to apply?
    
    I chose this model as the classification boundary is likely to be non linear and because we have 30 features which is much lower than our sample size.

In [158]:
# Train a model
# Support Vector Machine
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import svm
clf = svm.SVC()

# Fit model to training data
train_classifier(clf, 
                 X_train, 
                 y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.019
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)


In [159]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.013
F1 score for training set: 0.876068376068


In [160]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.783783783784


In [161]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    print "Test set size: {}".format(len(X_test))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf, X_train[:200], y_train[:200], X_test, y_test)
train_predict(clf, X_train[:100], y_train[:100], X_test, y_test)

------------------------------------------
Training set size: 300
Test set size: 95
Training SVC...
Done!
Training time (secs): 0.008
Predicting labels using SVC...
Done!
Prediction time (secs): 0.009
F1 score for training set: 0.876068376068
Predicting labels using SVC...
Done!
Prediction time (secs): 0.004
F1 score for test set: 0.783783783784
------------------------------------------
Training set size: 200
Test set size: 95
Training SVC...
Done!
Training time (secs): 0.006
Predicting labels using SVC...
Done!
Prediction time (secs): 0.020
F1 score for training set: 0.867924528302
Predicting labels using SVC...
Done!
Prediction time (secs): 0.007
F1 score for test set: 0.781456953642
------------------------------------------
Training set size: 100
Test set size: 95
Training SVC...
Done!
Training time (secs): 0.002
Predicting labels using SVC...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.877697841727
Predicting labels using SVC...
Done!
Prediction time (secs): 

### Random Forest Trees(RFT)

- What is the theoretical O(n) time & space complexity in terms of input size?

    Random Forest is an ensemble model of decision trees. Time complexity for building a complete unpruned decision tree is `O( f * n log(n) )`, where `n` is the sample size and `f` is the number of features. While building random forests you have to define the number of trees you want to build (assume it to be, `ntree`) and how many features you want to sample at each node (assume it to be, `ftry`).  Since we would only use `ftry` features at each node the complexity to build one tree would be `O( ftry * n log(n) )`. Now for building a random forest with `ntree` number of trees, the complexity would be `O( ntree * ftry * nlog(n) )` This is the worst case scenario, i.e., assuming the depth of the tree is going to be `O(log n)`. But in most cases the build process of a tree stops much before this and it is hard to estimate. But you could also restrict the depth of the trees that you would be building in your random forest. say you restrict the maximum depth of your tree to be `d` then the complexity calculations can be simplified to: `O( ntree * ftry * d * n)`
    
    The space complexity is `O(f n log(n))` with `f` is the number of features and `n` the number of elements in the dataset, under the assumption that a reasonably symmetric tree is built and that the evaluation of a single test takes constant time in the size of the dataset.

- What are the general applications of this model? What are its strengths and weaknesses?
    
    Some applications where it can be used are:
    
    In bioinformatics to study genes.
    In video tagging.
    In image classification.
    

    Strengths:
    Very good at handling binary features.
    Good at handling large volumes of data.

    Weaknesses:
    Tendency to overfit.
    Lower prediction accuracy.
    Tendency to have a high variance.
    

- Given what you know about the data so far, why did you choose this model to apply?
    
    I chose this model as the the features appear to be mutually exclusive from one onother and also because we have a non trivial amount of binary features.

In [125]:
# TODO: Train and predict using two other models
# Random Forest Trees
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10)

# Fit model to training data
train_classifier(clf, 
                 X_train, 
                 y_train)  # note: using entire training set here
print clf

Training RandomForestClassifier...
Done!
Training time (secs): 0.018
RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)


In [126]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.992700729927


In [127]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.763358778626


In [128]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    print "Test set size: {}".format(len(X_test))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf, X_train[:200], y_train[:200], X_test, y_test)
train_predict(clf, X_train[:100], y_train[:100], X_test, y_test)

------------------------------------------
Training set size: 300
Test set size: 95
Training RandomForestClassifier...
Done!
Training time (secs): 0.016
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.992665036675
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.666666666667
------------------------------------------
Training set size: 200
Test set size: 95
Training RandomForestClassifier...
Done!
Training time (secs): 0.015
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.992805755396
Predicting labels using RandomForestClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.802919708029
------------------------------------------
Training set size: 100
Test set size: 95
Training RandomForestClassifier...
Done!
Training time (secs): 0.015
Predicting labels using RandomForestC

### Gaussian Naive Bayes

- What is the theoretical O(n) time & space complexity in terms of input size?
    
    It has a time complexity of `O(nd+cd)` where `c` is the number of classes, `n` is the number of instances, `d` is the number of dimensions. It's space complexity is `O(dc)`.
    

- What are the general applications of this model? What are its strengths and weaknesses?
    
    Some applications where it can be used are:
    Classification of emails of spam.
    Classification of news stories into categories.
    
    Strengths:
    Very quick in terms of computation time.
    It can be used for both binary and multiclass classification problems.

    Weaknesses:
    Assumption of independence of attributes can be too constraining.
    

- Given what you know about the data so far, why did you choose this model to apply?
    
    I chose this model it makes it makes its classifications quickly, which is a desirable quality to have and also because some of the attributes seem to be mutually exclusive of each other.

In [129]:
# Combined code 
######################################### Train a model using clf.fit #########################################
# Gaussian Naive Bayes
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
# Fit model to training data
train_classifier(clf, 
                 X_train, 
                 y_train)  # note: using entire training set here
print clf  # you can inspect the learned model by printing it


######################################### Predict using clf.predict on training set ###############
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target, y_pred, pos_label='yes')

######################################### Compute F1 score ###############
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

######################################### Predict on test data #####################
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Training GaussianNB...
Done!
Training time (secs): 0.002
GaussianNB()
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.80378250591
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.763358778626


In [130]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    print "Test set size: {}".format(len(X_test))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant
train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf, X_train[:200], y_train[:200], X_test, y_test)
train_predict(clf, X_train[:100], y_train[:100], X_test, y_test)

------------------------------------------
Training set size: 300
Test set size: 95
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.80378250591
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.763358778626
------------------------------------------
Training set size: 200
Test set size: 95
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.002
F1 score for training set: 0.840579710145
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.724409448819
------------------------------------------
Training set size: 100
Test set size: 95
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.846715328467
Predic

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?

    The Support Vector Machine model is what I am using as the best model as it is the most appropriate and accurate model for the data we have at hand based on my analysis. It gave us the highest accuracy(f1 score) and was able to make the computations(both training and testing) in `0.017` seconds which is reasonably fast enough for our analysis. As far as resources and cost are concerned, SVMs are computationaly relatively light as they do not use the entire dataset to make their predictions and use only a subset of relevant data.


- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
    
    On a very high level, SVMs find a seperating line between data of two classes. Support vector machines are based on the concept of maximising the distance between datapoints of different classes and the seperating line. Think of it this way, if we have two kinds of chocolates and want to seperate them out using a ruler, we can do so placing the ruler in between the two types of chocolates and our objective is to increase the distance between the chocolate of that is closest to the ruler of one kind, and conversely between the chocolate of the second kind and the ruler. 
   
    Also, SVMs can be used to seperate data that cannot be separated using just a straight line using what is called a 'Kernel Trick' which is what I have used in my algorithm as well. I tuned my SVM model to get a higher F-1 score using the Grid Search technique.

- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.

- What is the model's final F<sub>1</sub> score?
    
    `0.794520547945`

In [164]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer
# Setup parameters and cross validation for model optimization through Grid Search
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]

svr = svm.SVC()
f1_scorer = make_scorer(f1_score, pos_label="yes")
# Define the Grid Search CV classifier
clf = GridSearchCV(svr, 
                   param_grid, 
                   scoring = f1_scorer)
train_classifier(clf, X_train, y_train)

# Best tuning
best_clf = clf.best_estimator_
print "The best estimator is: {}".format(best_clf)

# Train the model on the training data
train_classifier(best_clf, X_train, y_train)

# Test algorithm's performance using the test data
print "F1 score for training set: {}".format(predict_labels(best_clf, X_train, y_train))
print "F1 score for test set: {}".format(predict_labels(best_clf, X_test, y_test))

Training GridSearchCV...
Done!
Training time (secs): 145.355
The best estimator is: SVC(C=100, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.0001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)
Training SVC...
Done!
Training time (secs): 0.008
Predicting labels using SVC...
Done!
Prediction time (secs): 0.006
F1 score for training set: 0.832298136646
Predicting labels using SVC...
Done!
Prediction time (secs): 0.002
F1 score for test set: 0.794520547945


Bibliography:

http://www.jmlr.org/papers/volume6/tsang05a/tsang05a.pdf

http://www.cs.rutgers.edu/~mlittman/courses/ml04/svm.pdf

https://www.quora.com/What-is-the-time-complexity-of-Random-Forest-both-building-the-model-and-classification

https://www.quora.com/What-are-the-advantages-of-different-classification-algorithms

http://www.inf.ed.ac.uk/teaching/courses/iaml/slides/naive-2x2.pdf